In [1]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
import time
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler

Initialize the data points and Random centers
===========================


In [2]:
cluster=9

In [5]:
features=pd.read_csv('data.csv')

x= features.iloc[:,1:-1]
y=features.iloc[:,-1]
x=np.asarray(x)

scaler = StandardScaler()
x = scaler.fit_transform(np.array(x, dtype = float))

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)
print(np.unique(y))

[0 1 2 3 4 5 6 7 8 9]


In [7]:
def generateRandomCenters(c,x):
    centers=[]
    _min= (np.min(x))
    _max= (np.max(x))
    t= np.mean(x,axis=0)    
    for i in range(c):
        centers.append( t + 0.01*i)   
    return np.asarray(centers) 

generateRandomCenters(cluster,x)

array([[ 1.42108547e-16,  1.84741111e-16,  1.27897692e-16,
        -6.96331881e-16,  2.84217094e-17, -2.27373675e-16,
         4.26325641e-17,  7.10542736e-17, -1.77635684e-17,
         2.48689958e-17,  1.77635684e-17,  4.97379915e-17,
        -5.68434189e-17,  0.00000000e+00,  0.00000000e+00,
        -4.26325641e-17,  5.68434189e-17, -2.84217094e-17,
        -5.50670620e-17, -2.13162821e-17,  5.32907052e-17,
         7.10542736e-18, -6.39488462e-17,  2.13162821e-17,
        -4.26325641e-17, -2.13162821e-17],
       [ 1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02,
         1.00

In [8]:
# intialize random member ship
def initializeMembershipMatrix(n):
    membership_mat = list()
    for i in range(n.shape[0]):
        random_num_list = [random.random() for i in range(cluster)]
        summation = sum(random_num_list) 
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list) 
    return np.asarray(membership_mat) 

In [9]:
def updateMembershipValue3_2(u, v,val):
    distances = cdist(val, v,metric='euclidean')
    return distances

In [10]:
# def updateMembershipValue3_2(u, v,val):

#     for i in range(x.shape[0]):
#         distances=[]
#         for k in range(cluster):      
#             distances.append(np.linalg.norm(val[i]-v[k]) )     
#         u[i]=distances
        
#     return u

In [11]:

def CalculateWeightValues(cluster_center,val,q):
        
    membership=[]    
    distances = cdist(val, v,metric='euclidean')[:,q]  
    sumdistance=np.sum(distances)
    membership= distances/sumdistance 
    
    return membership


In [12]:

def CalculateWeightValuesByIndex(cluster_center,val,q,index):
    
#     print(centers[q])
#     distances=[]
#     for j in range(val.shape[0]):
#         distances.append(np.linalg.norm(val[j]- v[q])) 
    
    distances = cdist(val, v,metric='euclidean')[:,q]  
    sumdistance=np.sum(distances)
    membership= distances[index]/sumdistance
        
    return membership


In [13]:
def calcError(center,mem):
    dist=0
    for i in range(mem.shape[0]):
        dist+=np.linalg.norm(center - mem[i])
    return dist

In [14]:
def moveVtoTheCenter(v,u,x):
    for k in range( cluster):
        items=[]
        for i  in range(  (x.shape[0])):
            if(u[i,k]>=np.max(u[i])):
                items.append(x[i])
        v[k]=np.mean(items, axis=0)
    return v

In [15]:
def _k_init(X, n_clusters, x_squared_norms, random_state, n_local_trials=None):
    """Init n_clusters seeds according to k-means++
    Parameters
    ----------
    X : array or sparse matrix, shape (n_samples, n_features)
        The data to pick seeds for. To avoid memory copy, the input data
        should be double precision (dtype=np.float64).
    n_clusters : integer
        The number of seeds to choose
    x_squared_norms : array, shape (n_samples,)
        Squared Euclidean norm of each data point.
    random_state : int, RandomState instance
        The generator used to initialize the centers. Use an int to make the
        randomness deterministic.
        See :term:`Glossary <random_state>`.
    n_local_trials : integer, optional
        The number of seeding trials for each center (except the first),
        of which the one reducing inertia the most is greedily chosen.
        Set to None to make the number of trials depend logarithmically
        on the number of seeds (2+log(k)); this is the default.
    Notes
    -----
    Selects initial cluster centers for k-mean clustering in a smart way
    to speed up convergence. see: Arthur, D. and Vassilvitskii, S.
    "k-means++: the advantages of careful seeding". ACM-SIAM symposium
    on Discrete algorithms. 2007
    Version ported from http://www.stanford.edu/~darthur/kMeansppTest.zip,
    which is the implementation used in the aforementioned paper.
    """
    n_samples, n_features = X.shape

    centers = np.empty((n_clusters, n_features), dtype=X.dtype)

    assert x_squared_norms is not None, 'x_squared_norms None in _k_init'

    # Set the number of local seeding trials if none is given
    if n_local_trials is None:
        # This is what Arthur/Vassilvitskii tried, but did not report
        # specific results for other than mentioning in the conclusion
        # that it helped.
        n_local_trials = 2 + int(np.log(n_clusters))

    # Pick first center randomly
    center_id = random_state.randint(n_samples)
    if sp.issparse(X):
        centers[0] = X[center_id].toarray()
    else:
        centers[0] = X[center_id]

    # Initialize list of closest distances and calculate current potential
    closest_dist_sq = euclidean_distances(
        centers[0, np.newaxis], X, Y_norm_squared=x_squared_norms,
        squared=True)
    current_pot = closest_dist_sq.sum()

    # Pick the remaining n_clusters-1 points
    for c in range(1, n_clusters):
        # Choose center candidates by sampling with probability proportional
        # to the squared distance to the closest existing center
        rand_vals = random_state.random_sample(n_local_trials) * current_pot
        candidate_ids = np.searchsorted(stable_cumsum(closest_dist_sq),
                                        rand_vals)

        # Compute distances to center candidates
        distance_to_candidates = euclidean_distances(
            X[candidate_ids], X, Y_norm_squared=x_squared_norms, squared=True)

        # Decide which candidate is the best
        best_candidate = None
        best_pot = None
        best_dist_sq = None
        for trial in range(n_local_trials):
            # Compute potential when including center candidate
            new_dist_sq = np.minimum(closest_dist_sq,
                                     distance_to_candidates[trial])
            new_pot = new_dist_sq.sum()

            # Store result if it is the best local trial so far
            if (best_candidate is None) or (new_pot < best_pot):
                best_candidate = candidate_ids[trial]
                best_pot = new_pot
                best_dist_sq = new_dist_sq

        # Permanently add best center candidate found in local tries
        if sp.issparse(X):
            centers[c] = X[best_candidate].toarray()
        else:
            centers[c] = X[best_candidate]
        current_pot = best_pot
        closest_dist_sq = best_dist_sq

    return centers

In [16]:
# x_squared_norms = np.row_norms(x, squared=True)

from sklearn.utils.extmath import row_norms
from sklearn.utils.extmath import row_norms, squared_norm, stable_cumsum
import scipy.sparse as sp
from sklearn.metrics.pairwise import euclidean_distances
 
c = _k_init(x,cluster, row_norms(x, squared=True), np.random.RandomState())
c

array([[ 1.71545165e+00, -6.35896579e-01,  1.05952008e+00,
         5.99900284e-01,  7.81142398e-01,  4.12847441e-01,
        -7.63324886e-01, -1.86542631e+00, -2.34165047e-01,
        -1.60583733e-01,  1.09112321e+00, -8.19798412e-02,
         1.11531188e+00, -2.37879351e-01,  1.48397160e+00,
         3.45678596e-01,  1.58239438e+00,  3.01980182e-01,
         1.83009684e+00,  7.34517676e-01,  1.32239045e+00,
        -6.08304375e-01,  5.31525052e-01,  6.66743009e-01,
         1.93340950e+00,  1.25442382e+00],
       [ 2.03941442e-01, -2.18334463e-01, -5.91989337e-01,
        -1.30333985e-01, -4.76540849e-01, -1.02162458e+00,
        -7.26587903e-02,  1.77835030e-01,  1.26771421e-01,
         6.99892734e-01,  1.02246770e+00,  3.85751693e-02,
         1.68735345e+00, -3.35376401e-01,  1.66870645e+00,
        -6.71408490e-01,  7.32360368e-01, -6.98540788e-01,
         6.73790996e-01, -1.16092731e+00,  4.40963052e-01,
        -3.96850954e-01, -6.62285581e-01, -7.59261081e-01,
         3.06

Initialization
=================

In [17]:
import random
# x =np.asarray( random.sample(x.tolist(), len(x.tolist())))
# print(x[:5,])

u=initializeMembershipMatrix(x)
v=c#generateRandomCenters(cluster,x)
 
w=updateMembershipValue3_2(u,v,x)


ny=np.argmin(w,axis=1)

# for i in range ( cluster):
#     plt.scatter(x[ny == i, 0], x[ny == i, 1],  s=10, label="Cluster - "+str(i))
# plt.scatter(x=np.asarray(v)[:,0],y=np.asarray(v)[:,1], color='black',s=50)
# plt.show()  

POCS -- Parallel projection
=================

In [18]:
# val=x
errorlist=[]
start=time.time()
for t in range(50):
    print(t)
    error=0
    for q in range(cluster): 
         
        val=x[ny == q,:] 
                
        if(val.shape[0]!=0):
            
            weight=CalculateWeightValues(v[q],val,q) 

            temp=0    
            for i in range(val.shape[0]):                
                temp += weight[i]*(val[i]- v[q])
            v[q]= v[q]+temp    

        error+=calcError(v[q],val)
    
#     print(t,q,error) 
    errorlist.append([error,time.time()- start])

    w=updateMembershipValue3_2(u,v,x)  
    ny=np.argmin(w,axis=1)
    
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [19]:

from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics.cluster import normalized_mutual_info_score

print(adjusted_rand_score(y,ny))
print(mean_squared_error(y,ny))
print(normalized_mutual_info_score(y,ny))


0.18216173035905106
17.901
0.2957504190788666


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
